Run scripts to create classifier from AFKABAN data.

In [13]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier
import hyperopt
from hyperopt import tpe
from hpsklearn import HyperoptEstimator, k_neighbors_classifier, svc, lightgbm_classification, gaussian_nb, any_preprocessing,min_max_scaler, normalizer
import lightgbm
from datetime import timedelta, date
from tenacity import retry, stop_after_attempt
import AZKABANML
import sys, errno


# Read the dataframes

In [14]:
# Set paths
path = 'F:/AFKABAN/review2024/'
classifypath = 'F:/AFKABAN/Classify/'

df_120 = pd.read_feather(f'{path}/df_120_balanced.feather')
df_200 = pd.read_feather(f'{path}/df_200_balanced.feather')

df_120_sigbs = pd.read_feather(f'{path}/df_120_sigbs_balanced.feather')
df_200_sigbs = pd.read_feather(f'{path}/df_200_sigbs_balanced.feather')

track_120 = pd.read_feather(f'{path}/track_120_balanced.feather')
track_200 = pd.read_feather(f'{path}/track_200_balanced.feather')

# Run different iterations


## Preprocessing
Any! Give better results (+~0.2 on mean F1 score), either scale or min,max scaler or PCA.

In [15]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/AFKABAN/review2024/'
classifypath = 'F:/AFKABAN/Classify/'
d1 = date.today().strftime("%d-%m-%Y")

# NESTED CROSS-VALIDATION
#preprocessing = [min_max_scaler(unique_id)] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 50 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 3 # No. of times to retry before failing

### kNN

In [20]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+d1+'_120' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_120, clf, unique_id, classifypath, track_120, preprocessing=[])

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:02<00:00,  2.43s/trial, best loss: 0.010123627506327226]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.14s/trial, best loss: 0.00906499134987282]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:03<00:00,  3.52s/trial, best loss: 0.010655027215582313]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.43s/trial, best loss: 0.010133144150782258]
Beginning fold 5 of 10
 98%|█████████▊| 47/48 [00:00<?, ?trial/s, best loss=?]

In [ ]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+d1+'_200' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_200, clf, unique_id, classifypath, track_200, preprocessing=[])

In [17]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+d1+'_120_norm' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_120, clf, unique_id, classifypath, track_120, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:02<00:00,  2.66s/trial, best loss: 0.029352220189054812]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.90s/trial, best loss: 0.03368614992026131]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:03<00:00,  3.02s/trial, best loss: 0.03686654063749195]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.39s/trial, best loss: 0.03208731253695507]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:03<00:00,  3.95s/trial, best loss: 0.03313393796208319]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:02<00:00,  2.32s/trial, best loss: 0.029936783895862518]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:03<00:00,  3.20s/trial, best loss: 0.0315260084169664]
Beginning fold 8 of 10
100%|██████████| 50/50 [00:03<00:00,  3.08s/trial, best loss: 0.034146485749176225]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:02<00:00,  2.13s/trial, best loss: 0.031006928746433515]
Beginning fold 10 of 10
100%|██████████| 50/50 [00:02<00:00,  2.29

In [18]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+ d1 +'_200_norm' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_200, clf, unique_id, classifypath, track_200, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:02<00:00,  2.59s/trial, best loss: 0.027065044834090335]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:03<00:00,  3.03s/trial, best loss: 0.02873554085309582]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:02<00:00,  2.58s/trial, best loss: 0.031364892227279006]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.29s/trial, best loss: 0.03137595191179943]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:01<00:00,  1.98s/trial, best loss: 0.026548132445350703]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:02<00:00,  2.36s/trial, best loss: 0.034505596740026956]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:02<00:00,  2.88s/trial, best loss: 0.029198522298931606]
Beginning fold 8 of 10
100%|██████████| 50/50 [00:02<00:00,  2.35s/trial, best loss: 0.027588959591281936]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:02<00:00,  2.11s/trial, best loss: 0.0302422783694547]
Beginning fold 10 of 10
100%|██████████| 50/50 [00:02<00:00,  2.

#### lightGBM

In [ ]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120_norm_w_h' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify_test(df_120_w_h, WBAT_df_120, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [01:02<00:00, 62.57s/trial, best loss: 0.12777777777777777]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 17/17 [00:09<00:00,  9.85s/trial, best loss: 0.13043478260869568]


100%|███████████████████████████████████████████████| 50/50 [00:15<00:00, 15.30s/trial, best loss: 0.12673450508788164]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [02:14<00:00, 134.03s/trial, best loss: 0.13506012950971324]


100%|███████████████████████████████████████████████| 50/50 [00:45<00:00, 45.75s/trial, best loss: 0.13043478260869568]
Beginning fold 4 of 10
100%|████████████████████████████████████████████████| 50/50 [00:27<00:00, 27.83s/trial, best loss: 0.1285846438482886]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 2/2 [01:39<00:00, 99.21s/trial, best loss: 0.15726179463459755]


100%|███████████████████████████████████████████████| 50/50 [01:33<00:00, 93.48s/trial, best loss: 0.13506012950971324]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:48<00:00, 48.39s/trial, best loss: 0.14338575393154485]


100%|███████████████████████████████████████████████| 50/50 [00:10<00:00, 10.92s/trial, best loss: 0.14338575393154485]
Beginning fold 7 of 10
100%|████████████████████████████████████████████████| 38/38 [00:41<00:00, 41.72s/trial, best loss: 0.1369102682701202]


100%|████████████████████████████████████████████████| 50/50 [00:50<00:00, 50.17s/trial, best loss: 0.1369102682701202]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:40<00:00, 40.71s/trial, best loss: 0.13043478260869568]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:20<00:00, 20.58s/trial, best loss: 0.14801110083256241]


100%|████████████████████████████████████████████████| 50/50 [01:01<00:00, 61.57s/trial, best loss: 0.1258094357076781]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 23/23 [00:27<00:00, 27.34s/trial, best loss: 0.13320999074930617]


100%|███████████████████████████████████████████████| 50/50 [00:36<00:00, 36.01s/trial, best loss: 0.13320999074930617]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 41/41 [04:09<00:00, 249.85s/trial, best loss: 0.13072439633638633]


100%|███████████████████████████████████████████████| 50/50 [01:18<00:00, 78.20s/trial, best loss: 0.13072439633638633]
Classifying new data


In [ ]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200_norm_w_h' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify_test(df_200_w_h, WBAT_df_200, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [01:22<00:00, 82.40s/trial, best loss: 0.12084870848708484]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 17/17 [00:26<00:00, 26.84s/trial, best loss: 0.10885608856088558]


100%|███████████████████████████████████████████████| 50/50 [05:04<00:00, 304.75s/trial, best loss: 0.1060885608856088]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:21<00:00, 21.30s/trial, best loss: 0.11254612546125464]


 97%|████████████████████████████████████████████████████████████████████████  | 38/39 [00:00<?, ?trial/s, best loss=?]

In [ ]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+d1+'_120_norm' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_test(df_120, WBAT_df_120, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  5.00s/trial, best loss: 0.027185501066098072]
Beginning fold 2 of 10
100%|████████████████████████████████████████████████| 17/17 [00:06<00:00,  6.10s/trial, best loss: 0.0357142857142857]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.89s/trial, best loss: 0.02985074626865669]
Beginning fold 3 of 10
 96%|███████████████████████████████████████████████████████████████████████▏  | 25/26 [00:00<?, ?trial/s, best loss=?]

job exception: C <= 0



 96%|███████████████████████████████████████████████████████████████████████▏  | 25/26 [00:04<?, ?trial/s, best loss=?]
Resuming cross-validation from fold 3
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.27s/trial, best loss: 0.04104477611940294]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 16/16 [00:05<00:00,  5.22s/trial, best loss: 0.028784648187633266]


100%|██████████████████████████████████████████████| 50/50 [00:46<00:00, 46.37s/trial, best loss: 0.028784648187633266]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 34/34 [00:28<00:00, 28.07s/trial, best loss: 0.03038379530916846]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.69s/trial, best loss: 0.029317697228145034]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 50/50 [02:34<00:00, 154.69s/trial, best loss: 0.03090037293553538]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.34s/trial, best loss: 0.5503462972828983]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.29s/trial, best loss: 0.031965903036760834]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 19/19 [00:05<00:00,  5.90s/trial, best loss: 0.03516249334043686]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.80s/trial, best loss: 0.03143313798614811]
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 37/37 [00:05<00:00,  5.58s/trial, best loss: 0.03090037293553538]


100%|███████████████████████████████████████████████| 50/50 [00:28<00:00, 28.22s/trial, best loss: 0.03090037293553538]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.45s/trial, best loss: 0.029302077783697422]
Retraining model on full dataset
100%|█████████████████████████████████████████████████| 4/4 [00:57<00:00, 57.22s/trial, best loss: 0.04412470023980819]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.71s/trial, best loss: 0.03597122302158273]
Classifying new data


In [ ]:
# CLASSIFIER SVC 200
unique_id = 'SVC_'+d1+'_200_norm' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_test(df_200, WBAT_df_200, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.41s/trial, best loss: 0.03868574456809748]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [00:05<00:00,  5.71s/trial, best loss: 0.038155802861685184]


100%|█████████████████████████████████████████████| 50/50 [01:51<00:00, 111.79s/trial, best loss: 0.034446210916799114]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [00:05<00:00,  5.53s/trial, best loss: 0.035506094329623705]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.91s/trial, best loss: 0.035506094329623705]
Beginning fold 4 of 10
 96%|███████████████████████████████████████████████████████████████████████▎  | 27/28 [00:00<?, ?trial/s, best loss=?]

## Linear
Balanced + PP

In [ ]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_120_sigbs, clf, unique_id, classifypath, preprocessing=[min_max_scaler(unique_id)])

In [ ]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier

AZKABANML.main_classify(df_200_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

In [ ]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_120_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

In [ ]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_200_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

In [ ]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify(df_120_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

In [ ]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify(df_200_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))